In [1]:
import json 
import csv
import torch
import numpy as np
from transformers import BertTokenizer,BertModel
from torch import nn
from torch.optim import Adam
from tqdm.notebook import tqdm

td_route = "./dm2021-lab2-hw2"

In [2]:
Tweet_list = []
with open(td_route + '/tweets_DM.json') as f:
    for jsonObj in tqdm(f):
        Tweet_list.append(json.loads(jsonObj))

0it [00:00, ?it/s]

In [3]:
ori_Tweet_list = Tweet_list

In [4]:
if(True):
    Tweet_list = ori_Tweet_list[len(ori_Tweet_list)//10*0:len(ori_Tweet_list)//10*5]

In [5]:
identification = {'test':set(),'train':set()}
with open(td_route + '/data_identification.csv', newline='') as csvfile:
    rd = csv.reader(csvfile, delimiter=',', quotechar='|')
    for tid,ident in rd:
        if(ident in identification):identification[ident].add(tid)

In [6]:
emotions = {}
with open(td_route + '/emotion.csv', newline='') as csvfile:
    rd = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(rd, None) #Skip header
    for tid,emt in rd:
        emotions[tid] = emt

In [7]:
Tweet_dict = {'train':{},'test':{}}
for tl in Tweet_list:
    tid = tl['_source']['tweet']['tweet_id']
    if(tid in identification['train']):
        Tweet_dict['train'][tid] = tl['_source']['tweet']['text']
    elif(tid in identification['test']):
        Tweet_dict['test'][tid] = tl['_source']['tweet']['text']

In [8]:
print(len(Tweet_dict['train']),len(Tweet_dict['test']))
print(len(emotions))
print(len(Tweet_list),len(ori_Tweet_list))

727734 206031
1455563
933765 1867535


In [9]:
#tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')    # Download vocabulary from S3 and cache
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [10]:
#model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased', output_attentions=True)  # Update configuration during loading

In [11]:
#indexed_tokens = tokenizer.encode(Tweet_dict['train']['0x376b20'], add_special_tokens=True)
tokenz = tokenizer(Tweet_dict['train']['0x376b20'],padding='max_length', max_length = 280, truncation=True,return_tensors="pt")
tokenz

{'input_ids': tensor([[  101,  2563,  1150,  2112,   107,  5194,  1143,  1113,   108,   156,
         27864,  7147,  1204,   107,  1538,  1129,  1260,  7889,  7412,  1906,
           119,   140, 14875,  1299,   119,   119,   119,   119,  1115,   112,
           188,   133,   149,  3048,   135,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [12]:
labels = {'anger':0,'anticipation':1,'disgust':2,'fear':3,'sadness':4,'surprise':5,'trust':6,'joy':7}

In [13]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts):
        self.labels = [labels[emotions[key]] for key in tqdm(texts)]
        self.texts = [tokenizer(texts[key],padding='max_length', max_length = 512, truncation=True,return_tensors="pt") for key in tqdm(texts)]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return self.labels[idx]

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [14]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 8)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [15]:
full_ds = Dataset(Tweet_dict['train'])
#train_ds,test_ds = random_split(full_ds, [3, 7], generator=torch.Generator().manual_seed(42))

  0%|          | 0/727734 [00:00<?, ?it/s]

  0%|          | 0/727734 [00:00<?, ?it/s]

In [16]:
#torch.save(full_ds, 'dataset.pt')

In [17]:
train_size = int(len(full_ds)*0.7)
test_size = int(len(full_ds)*0.3)
if(len(full_ds) % 10 != 0):test_size += 1
train_ds,test_ds = torch.utils.data.random_split(full_ds, [train_size, test_size], generator=torch.Generator().manual_seed(42069))

In [18]:
print(len(train_ds),len(test_ds),len(train_ds)/len(full_ds),len(test_ds)/len(full_ds))
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=2)

509413 218321 0.6999989006972328 0.3000010993027672


In [19]:
#Operation Checks
batch_iterator = iter(train_dataloader)
inputs, label = next(batch_iterator)
#for inputs,label in tqdm(batch_iterator):
    #continue
print(label)
del batch_iterator

tensor([7, 3])


In [20]:
def train(model, learning_rate, epochs):

    #train, val = Dataset(train_data), Dataset(val_data)
    train,val = train_ds,test_ds

    use_cuda = torch.cuda.is_available()
    #use_cuda = False
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    cnter = 12

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.type(torch.LongTensor).to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                #print(output,train_label)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in tqdm(val_dataloader):

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_ds): .3f} \
                | Train Accuracy: {total_acc_train / len(train_ds): .3f} \
                | Val Loss: {total_loss_val / len(test_ds): .3f} \
                | Val Accuracy: {total_acc_val / len(test_ds): .3f}')
            torch.save(model.state_dict(), 'best_checkpoint_1222_'+ str(cnter) +'.pth')
            cnter += 1

In [23]:
EPOCHS = 1
#model = BertClassifier()
#model.load_state_dict(torch.load('best_checkpoint_1222_11.pth'))
LR = 1e-6

In [24]:
EPOCHS = 1
train(model, LR, EPOCHS)

  0%|          | 0/254707 [00:00<?, ?it/s]

  0%|          | 0/109161 [00:00<?, ?it/s]

Epochs: 1 | Train Loss:  0.430                 | Train Accuracy:  0.701                 | Val Loss:  0.428                 | Val Accuracy:  0.701


In [25]:
!nvidia-smi

Thu Dec 30 01:49:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 28%   40C    P8     9W / 180W |   5293MiB /  8192MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
Tweet_list2 = []
with open(td_route + '/tweets_DM.json') as f:
    for jsonObj in tqdm(f):
        Tweet_list2.append(json.loads(jsonObj))

0it [00:00, ?it/s]

In [29]:
test_dict = {}
for tl in ori_Tweet_list:
    tid = tl['_source']['tweet']['tweet_id']
    if(tid in identification['test']):
        test_dict[tid] = tl['_source']['tweet']['text']
print(len(test_dict))

411972


In [28]:
rev_labels = {}
for key in labels:
    rev_labels[labels[key]] = key

In [30]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
outs = []
with torch.no_grad():
    for key in tqdm(test_dict):
        val_input = tokenizer(test_dict[key],padding='max_length', max_length = 280, truncation=True,return_tensors="pt")
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)
        output = model(input_id, mask).to('cpu')
        #print(torch.argmax(output))
        outs.append([key,rev_labels[int(torch.argmax(output))]])

  0%|          | 0/411972 [00:00<?, ?it/s]

In [31]:
import csv
print(len(outs))
with open('val3.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["id"]+["emotion"])
    for row in outs:  
        spamwriter.writerow(row)

411972


411972
